In [77]:
import pandas as pd

tripdata = pd.read_csv('../data/citibike-tripdata.csv', sep=',')
tripdata.head(2)

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0


In [17]:
# Сколько пропусков в столбце start station id?
tripdata['start station id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 300000 entries, 0 to 299999
Series name: start station id
Non-Null Count   Dtype  
--------------   -----  
299831 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB


In [18]:
# Какой тип данных имеют столбцы starttime и stoptime?
tripdata.iloc[0:][['starttime', 'stoptime']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   starttime  300000 non-null  object
 1   stoptime   300000 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


In [21]:
# Найдите идентификатор самой популярной стартовой стоянки
tripdata['start station id'].mode()

0    281.0
Name: start station id, dtype: float64

In [22]:
# Велосипед с каким идентификатором является самым популярным?
tripdata['bikeid'].mode()

0    33887
Name: bikeid, dtype: int64

In [25]:
# Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
tripdata['usertype'].value_counts(normalize=True)

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

In [26]:
# Кто больше занимается велоспортом — мужчины или женщины?
tripdata['gender'].value_counts()

1    183582
2     74506
0     41912
Name: gender, dtype: int64

In [69]:
# Проверить гипотезы:
# A Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу
print(tripdata['start station id'].nunique() != tripdata['end station id'].nunique())
# B В рассматриваемые дни минимальный возраст клиента составлял 10 лет
print(2022 - tripdata['birth year'].min() == 10)
# C Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave"
print(tripdata['start station name'].value_counts().nsmallest(1))
# D Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail"
print(tripdata['end station name'].mode())


True
False
Eastern Pkwy & Washington Ave    6
Name: start station name, dtype: int64
0    West St & Chambers St
Name: end station name, dtype: object


In [78]:
# Присутствуют столбцы, которые дублируют информацию друг о друге: идентификаторы и названия стартовой и конечной стоянки.
# Удалите признаки идентификаторов стоянок
tripdata.drop(['start station id','end station id'], axis=1, inplace=True)
tripdata.shape

(300000, 12)

In [79]:
# Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. 
# Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?
tripdata['age'] = 2018 - tripdata['birth year']
tripdata.drop(['birth year'], axis=1, inplace=True)
tripdata[tripdata['age']>60].shape[0]

11837

In [80]:
# Создайте признак длительности поездки trip duration - интервал времени между временем окончания поездки (stoptime) 
# и её началом (starttime). Сколько целых минут длилась поездка под индексом 3 в таблице?
tripdata['trip duration'] = pd.to_datetime(tripdata['stoptime'], dayfirst=False) - pd.to_datetime(tripdata['starttime'], dayfirst=False)
tripdata.iloc[3]['trip duration'].seconds/60

In [102]:
# Создайте признак weekend, который равен 1, если поездка начиналась в выходной день, и 0 — в противном случае. 
# Выясните, сколько поездок начиналось в выходные.
tripdata['starttime'] = pd.to_datetime(tripdata['starttime'])
tripdata['stoptime'] = pd.to_datetime(tripdata['stoptime'])
tripdata['weekend'] = tripdata['starttime'].apply(lambda x: 1 if x.dayofweek == 5 or x.dayofweek == 6 else 0)
tripdata[tripdata['weekend'] == 1].shape[0]

115135

In [106]:
# Создайте признак времени суток поездки time_of_day из часа начала поездки:
# - поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# - поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# - поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# - поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
# Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью?
def calc_time_of_day(starttime):
    hour = starttime.hour
    if hour >= 0 and hour <= 6:
        result = 'night'
    elif hour > 6 and hour <= 12:
        result = 'morning'
    elif hour > 12 and hour <= 18:
        result = 'day'
    else:
        result = 'evening' 
    return result

tripdata['time_of_day'] = tripdata['starttime'].apply(calc_time_of_day)
tripdata[tripdata['time_of_day'] == 'day'].shape[0]/tripdata[tripdata['time_of_day'] == 'night'].shape[0]


9.480411004308916